
## Toxic Comment Classification Challenge
<b>Authors:</b> Yohannis Telila, Dawit Anelay<br>
<b>Date :</b> 12/04/2021<br>
<b>Course:</b> Human Language Technologies, 2020/2021 A.Y<br>
<b>Lecturer:</b> Prof. Giuseppe Attardi<br>
<b>data set source :</b> https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data


## 1. Preprocessing and EDA
### 1.1 Understanding data
// Detailed comment coming up . . .

Note : The data is highly unbalanced. read more about it.

In [32]:
# importing necessary libraries
import pandas as pd
import numpy as np

In [33]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
testLabel = pd.read_csv('data/test_labels.csv')
sample = pd.read_csv('data/sample_submission.csv')

In [34]:
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [35]:
train.head(70)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
65,0028d62e8a5629aa,All of my edits are good. Cunts like you who ...,1,0,1,0,1,0
66,00290e2a171dd073,"""\n - NeilN's bang on - You aren't being haras...",0,0,0,0,0,0
67,002918ae66cc4bc2,"I went there around the same time he did, and ...",0,0,0,0,0,0
68,0029541a38c523a0,There must be some chemical imbalance in your ...,0,0,0,0,0,0


<p>We have our comments in <i>comment_text</i> column and comment labels in binarized format(one hot encoded format)</p>    

In [36]:
train.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [37]:
train.shape

(159571, 8)

In [102]:

print("toxic:", np.count_nonzero(train[['toxic']]))
print("severe_toxic:", np.count_nonzero(train[['severe_toxic']]))
print("threat:",np.count_nonzero(train[['threat']]))
print("identity_hate:",np.count_nonzero(train[['identity_hate']]))

toxic: 15294
severe_toxic: 1595
threat: 478
identity_hate: 1405


In [38]:
train['comment_text'][3]

'"\nMore\nI can\'t make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of ""types of accidents""  -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It\'s listed in the relevant form eg Wikipedia:Good_article_nominations#Transport  "'

### 1.2 Cleaning 

// Detailed comment coming up . . .<br>
// My thought : We can also remove numbers from comment since they dont express any emotion()

In [49]:
import re

def clean_comment(comment):
    """
        A function to clean the dataset 
        precise comment coming up 
    """
    comment = comment.lower()
    comment = re.sub(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', '', comment) # clean url
    comment = re.sub(r'(jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)','',comment) # clean month
    comment = re.sub(r'\d+', '', comment)                           # clean numbers
    comment = re.sub(r'[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+','',comment) # clean email 
    comment = re.sub(r'<[^>]+>', '', comment)                       # clean tage
    comment = re.sub(r'@(\w+)', '', comment)                        # clean @
    comment = re.sub(r'\n', '', comment)                            # clean \n
    comment = re.sub(r'#(\w+)', '', comment)                        # clean hashes
    comment = re.sub(r'([^\x00-\x7F])+','',comment)                 # clean Non-English characters 
    comment =  re.sub(r'[,(\)!@\"?\.$%_&#*+-:;]', '', comment)      # clean punctuation including parenthesis and excluding '
    
    return comment 

In [50]:
train['comment_text'] = train['comment_text'].apply(clean_comment)
test['comment_text'] = test['comment_text'].apply(clean_comment)

In [51]:
train['comment_text'][70]

"oppose  as the article stands it does not properly reference under the wpreference guidelines if this issue is resolved i don't any issues with it being moved to the mainspace "

In [14]:
test['comment_text'][3]

"if you have a look back at the source the information i updated was the correct form i can only guess the source hadn't updated i shall update the information once again but thank you for your message"

### 1.3 Expanding contracted comments.

// Detailed comment coming up . . .


In [15]:
import json
# contracted words list with their expanded form.    # can't to can not 
with open('expandedContractions.json') as json_file:
    cList = json.load(json_file)

In [16]:
def expandComment(x):
    if type(x) is str:
        for key in cList:
            value = cList[key]
            x = x.replace(key, value)
        return x
    else:
        return x

In [17]:
train['comment_text'] = train['comment_text'].apply(lambda x: expandComment(x))
test['comment_text'] = test['comment_text'].apply(lambda x: expandComment(x))

In [18]:
train.head(50)["comment_text"][3] 

'\nmore\ni cannot make any real suggestions on improvement  i wondered if the section statistics should be later on or a subsection of types of accidents  i think the references  need tidying so that they are all in the exact same format ie date format etc i can do that later on if noone else does first  if you have any preferences for formatting style on references or want to do it yourself please let me know\n\nthere appears to be a backlog on articles for review so i guess there  be a delay until a reviewer turns up it is listed in the relevant form eg wikipediagoodarticlenominations  '

### 1.4 Removing Stop Words (or/and Frequent words/ Rare words):
Filtering out unnecessary words. <br>
// More detail comments are coming up.

In [19]:
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [21]:
train['comment_text'][3]

'\nmore\ni cannot make any real suggestions on improvement  i wondered if the section statistics should be later on or a subsection of types of accidents  i think the references  need tidying so that they are all in the exact same format ie date format etc i can do that later on if noone else does first  if you have any preferences for formatting style on references or want to do it yourself please let me know\n\nthere appears to be a backlog on articles for review so i guess there  be a delay until a reviewer turns up it is listed in the relevant form eg wikipediagoodarticlenominations  '

In [22]:
train['comment_text'] = train['comment_text'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop))
test['comment_text'] = test['comment_text'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop))

In [23]:
train['comment_text'][3]

'cannot make real suggestions improvement wondered section statistics later subsection types accidents think references need tidying exact format ie date format etc later noone else first preferences formatting style references want please let know appears backlog articles review guess delay reviewer turns listed relevant form eg wikipediagoodarticlenominations'

### 1.5 Stemming
For stemming we are going to use PorterStemmer <br>

// Detailed comment coming up . . .

In [117]:
from nltk.stem import PorterStemmer

def porter_stemmer(text):
    """
        Stem words in list of tokenized words with PorterStemmer
    """
    stemmer = nltk.PorterStemmer()
    stems = [stemmer.stem(i) for i in text]
    return stems

In [20]:
"""
 
 I am getting unexpected output here. I will just comment it out for now.

"""
#test['comment_text'] = test['comment_text'].apply(lambda x: porter_stemmer(x))
#train['comment_text'] = train['comment_text'].apply(lambda x: porter_stemmer(x))

'\n \n I am getting unexpected output here. I will just comment it out for now.\n\n'

In [115]:
train['comment_text'][1]

"d'aww! he matches background colour i'm seemingly stuck with. thanks. (talk) :, , (utc)"

### 1.6 Separating comment and target

In [24]:
x_comTrain = train['comment_text']
y_comTrain = train[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]

In [31]:
x_comTrain[1]

"d'aww matches background colour seemingly stuck thanks talk utc"

vectorizing the comments.

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [3]:
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1)

In [ ]:
X_comm_vector = tfv.fit_transform(x_comTrain)

### 1.7 Splitting dataset to train and test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_comm_vector,y_comTrain, test_size=0.2, random_state=20)

In [ ]:
y_test.head()

In [1]:
X_train

NameError: name 'X_train' is not defined

## 2. Model 1 : Logistic regression
// More comments coming up. . . <br>

----> We gonna start with a simpler model <------- <br>

The main assumption here is that the labels are mutually exclusive. You do not consider any underlying correlation between the classes in this method.

For instance, it is more like asking simple questions, say, “is the comment toxic or not”, “is the comment threatening or not?”, etc. Also there might be an extensive case of overfitting here, since most of the comments are unlabeled, i,e., most of the comments are clean comments


### -> Model evaluation
For the model evaluation we gonna be using log loss. But later the evaluation matrics was changed to AUC. we gonna do both. 

> Exact Match Ratio (Subset accuracy):
It is the most strict metric, indicating the percentage of samples that have all their labels classified correctly.

> The disadvantage of this measure is that multi-class classification problems have a chance of being partially correct, but here we ignore those partially correct matches.
There is a function in scikit-learn which implements subset accuracy, called as accuracy_score.

We gonna test with <b>accuracy_score</b> for the time being.

In [30]:
# importing necessary libraries
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [32]:

categories = ['toxic', 'severe_toxic','obscene','threat','insult','identity_hate']


In [29]:

LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
            ])

for category in categories:
    print('Training--> {0} <-- comments.'.format(category))
    LogReg_pipeline.fit(X_train, y_train[category])
    prediction = LogReg_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(y_test[category], prediction)))
    print("\n")

Training--> toxic <-- comments.
Test accuracy is 0.9526554911483628


Training--> severe_toxic <-- comments.
Test accuracy is 0.9903806987310042


Training--> obscene <-- comments.
Test accuracy is 0.9740247532508225


Training--> threat <-- comments.
Test accuracy is 0.9971173429421902


Training--> insult <-- comments.
Test accuracy is 0.9681341062196459


Training--> identity_hate <-- comments.
Test accuracy is 0.991540028199906




## 3. Model 2 - Binary relevance
> Model ref: https://github.com/scikit-multilearn/scikit-multilearn

In [30]:
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression

In [31]:
# initialize classifier chains multi-label classifier
classifier = ClassifierChain(LogisticRegression())
# Training logistic regression model on train data
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)



In [1]:
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")

NameError: name 'accuracy_score' is not defined


## 4. Model 3 - Näive Bayes Model

In [36]:
from sklearn.naive_bayes import MultinomialNB

In [37]:
NbReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(MultinomialNB(alpha = 1, fit_prior=True, class_prior=None), n_jobs=-1)),
            ])



for category in categories:
    print('Training--> {0} <-- comments.'.format(category))
    NbReg_pipeline.fit(X_train, y_train[category])
    Nb_prediction = NbReg_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(y_test[category], Nb_prediction)))
    print("\n")

Training--> toxic <-- comments.
Test accuracy is 0.9206642644524519


Training--> severe_toxic <-- comments.
Test accuracy is 0.989566034779884


Training--> obscene <-- comments.
Test accuracy is 0.9523108256305812


Training--> threat <-- comments.
Test accuracy is 0.9967726774244086


Training--> insult <-- comments.
Test accuracy is 0.9531568228105907


Training--> identity_hate <-- comments.
Test accuracy is 0.9903806987310042




## 3. svm

In [28]:
from sklearn.svm import SVC

In [35]:
Svc_Reg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(SVC(C=1.0, kernel='rbf',gamma='auto'), n_jobs=1)),
            ])

for category in categories:
    print('Training--> {0} <-- comments.'.format(category))
    Svc_Reg_pipeline.fit(X_train, y_train[category])
    Svc_prediction = Svc_Reg_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(y_test[category], Svc_prediction)))
    print("\n")


Training--> toxic <-- comments.
Test accuracy is 0.9024909916966943


Training--> severe_toxic <-- comments.
Test accuracy is 0.9901300328998903


Training--> obscene <-- comments.
Test accuracy is 0.9463888453705155


Training--> threat <-- comments.
Test accuracy is 0.997086009713301


Training--> insult <-- comments.
Test accuracy is 0.9506501644994517


Training--> identity_hate <-- comments.
Test accuracy is 0.9909446968510105




## 4. GRU

## 5. LSTM

## 6. xLNet
Ref. https://github.com/Shivampanwar/Bert-text-classification/blob/master/XLNet/xlnet_experimentation.ipynb
https://towardsdatascience.com/multi-label-text-classification-with-xlnet-b5f5755302df

## 7. Ensembling

## 8. Conclusion